## Setup and Imports


In [13]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from scipy.signal import fftconvolve
from scipy.io import loadmat

## Folder Structure Setup

In [3]:
# Using your existing folders
dry_audio_dir = ""
output_dir = ""
csv_path = "" # Estimated Acoustic Parameters CSV File Path
hrir_path = "" # HRIR Folder Path

os.makedirs(output_dir, exist_ok=True)
print("✅ Paths set.")

✅ Paths set.


## Loading HRIR Data

In [5]:
data = loadmat(hrir_path)
hrir_l = data['hrir_l']
hrir_r = data['hrir_r']

azimuths = np.array([-80.0, -65.0, -55.0, -45.0, -40.0, -35.0, -30.0, -25.0, -20.0, -15.0,
                     -10.0,  -5.0,   0.0,   5.0,  10.0,  15.0,  20.0,  25.0,  30.0,  35.0,
                      40.0,  45.0,  55.0,  65.0,  80.0])
azimuths = np.concatenate([azimuths[::-1] * -1, azimuths])[:50]
elevations = np.linspace(-45, 230.625, 25)

az_idx = np.argmin(np.abs(azimuths - 0.0))
el_idx = np.argmin(np.abs(elevations - 0.0))

hrir_left = hrir_l[el_idx, az_idx]
hrir_right = hrir_r[el_idx, az_idx]

print("✅ HRIR loaded for azimuth=0°, elevation=0°")

✅ HRIR loaded for azimuth=0°, elevation=0°


## Function Definition: fdn_reverb

In [7]:
def fdn_reverb(signal, sr, rt60, num_delays=4):
    delays = [149, 211, 263, 293][:num_delays]  # Prime numbers
    gains = [10 ** (-3 * d / (sr * rt60)) for d in delays]
    buffers = [np.zeros(d) for d in delays]
    out = np.zeros_like(signal)

    for n in range(len(signal)):
        sample = signal[n]
        feedback = sum(buffers[i][0] * gains[i] for i in range(num_delays)) / num_delays
        out[n] = sample + feedback
        for i in range(num_delays):
            buffers[i] = np.roll(buffers[i], -1)
            buffers[i][-1] = sample + feedback

    return out / np.max(np.abs(out))  # Normalize

## Function Definition: render_binaural_fdn

In [9]:
def render_binaural_fdn(dry_audio, sr, rt60, drr):
    wet = fdn_reverb(dry_audio, sr, rt60)
    drr_linear = 10 ** (drr / 20)
    mixed = dry_audio + (wet / drr_linear)

    left = fftconvolve(mixed, hrir_left, mode='full')[:len(mixed)]
    right = fftconvolve(mixed, hrir_right, mode='full')[:len(mixed)]
    binaural = np.stack([left, right], axis=1)

    return binaural / np.max(np.abs(binaural))

## Batch Processing and Rendering

In [11]:
df = pd.read_csv(csv_path).head(50)
dry_files = sorted([f for f in os.listdir(dry_audio_dir) if f.endswith(".wav")])[:len(df)]

print(f"✅ Found {len(dry_files)} dry files and {len(df)} parameter rows")

for i, filename in enumerate(dry_files):
    dry_path = os.path.join(dry_audio_dir, filename)
    dry_audio, sr = librosa.load(dry_path, sr=16000)

    row = df.iloc[i]
    rt60 = row['RT60']
    drr = row['DRR']
    c50 = row['C50']  # Not used

    print(f"🎧 [{i+1}/{len(dry_files)}] Processing {filename} | RT60={rt60:.2f}, DRR={drr:.2f}")
    rendered = render_binaural_fdn(dry_audio, sr, rt60, drr)

    out_path = os.path.join(output_dir, f"rendered_fdn_{i:03d}_{filename}")
    sf.write(out_path, rendered, sr)
    print(f"✅ Saved to {out_path}")

✅ Found 50 dry files and 50 parameter rows
🎧 [1/50] Processing sample_000.wav | RT60=0.56, DRR=-0.66
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs_fdn/rendered_fdn_000_sample_000.wav
🎧 [2/50] Processing sample_001.wav | RT60=0.87, DRR=-3.32
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs_fdn/rendered_fdn_001_sample_001.wav
🎧 [3/50] Processing sample_002.wav | RT60=0.78, DRR=0.89
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs_fdn/rendered_fdn_002_sample_002.wav
🎧 [4/50] Processing sample_003.wav | RT60=0.75, DRR=-7.55
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs_fdn/rendered_fdn_003_sample_003.wav
🎧 [5/50] Processing sample_004.wav | RT60=0.70, DRR=1.19
✅ Saved to /Users/petros/MSc Computer Science/EECS MSc Project 24-25/Approach_1/rendered_batch_outputs_fdn/render